In [1]:
import numpy as np
import sympy as smp
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import PillowWriter
from IPython.display import HTML

In [2]:
# Menentukan variabel yang diperlukan untuk sympy
t, m, g, L1, L2, w, C, alph, beta = smp.symbols(r't m g L_1, L_2 \omega C \alpha \beta')

In [3]:
#mendefinisikan theta 1 dan theta 2 dan menyatakan fungsi waktu. Juga mendefinisikan tuturan pertama dan kedua
the1, the2 = smp.symbols(r'\theta_1, \theta_2', cls=smp.Function)

In [4]:
the1 = the1(t)
the1_d = smp.diff(the1, t)
the1_dd = smp.diff(the1_d, t)

In [5]:
the2 = the2(t)
the2_d = smp.diff(the2, t)
the2_dd = smp.diff(the2_d, t)

In [6]:
# Mendeklarasikan nilai x1(theta1), y1(teta1), x2(teta1, teta2), y2(teta1, teta2)
x1, y1, x2, y2 = smp.symbols('x_1, y_1, x_2, y_2', cls=smp.Function)
x1 = x1(t, the1)
y1 = y1(t, the1)
x2 = x2(t, the1, the2)
y2 = y2(t, the1, the2)

In [7]:
# Masukkan ke dalam bentuk fungsional spesifik dari x1, y1, x2, y2
x1 = smp.cos(w * t) + L1 * smp.sin(the1)
y1 = -L1 * smp.cos(the1)
x2 = smp.cos(w * t) + L1 * smp.sin(the1) + L2 * smp.sin(the2)
y2 = -L1 * smp.cos(the1) - L2 * smp.cos(the2)

In [10]:
# Definisi fungsi numerik dari vx1, vy1, vx2, vy2
vx1_f = smp.lambdify((t, w, L1, L2, the1, the2, the1_d, the2_d), smp.diff(x1, t))
vx2_f = smp.lambdify((t, w, L1, L2, the1, the2, the1_d, the2_d), smp.diff(x2, t))
vy1_f = smp.lambdify((t, w, L1, L2, the1, the2, the1_d, the2_d), smp.diff(y1, t))
vy2_f = smp.lambdify((t, w, L1, L2, the1, the2, the1_d, the2_d), smp.diff(y2, t))

In [9]:
# Definisi fungsi numerik dari vx1, vy1, vx2, vy2
smp.diff(x1, t)

L_1*cos(\theta_1(t))*Derivative(\theta_1(t), t) - \omega*sin(\omega*t)

In [11]:
vx1_f = smp.lambdify((t, w, L1, L2, the1, the2, the1_d, the2_d), smp.diff(x1, t))
vx2_f = smp.lambdify((t, w, L1, L2, the1, the2, the1_d, the2_d), smp.diff(x2, t))
vy1_f = smp.lambdify((t, w, L1, L2, the1, the2, the1_d, the2_d), smp.diff(y1, t))
vy2_f = smp.lambdify((t, w, L1, L2, the1, the2, the1_d, the2_d), smp.diff(y2, t))

In [12]:
# Rumus lagrange
T = 1/2 * (smp.diff(x1, t)**2 + smp.diff(y1, t)**2) + 1/2 * m * (smp.diff(x2, t)**2 + smp.diff(y2, t)**2)
V = g * y1 + m * g * y2
L = T - V

In [13]:
# Persamaan Lagrange-Euler untuk theta1
LE1 = smp.diff(L, the1) - smp.diff(smp.diff(L, the1_d), t)
LE1 = LE1.simplify()

In [14]:
# Persamaan Lagrange-Euler untuk theta2
LE2 = smp.diff(L, the2) - smp.diff(smp.diff(L, the2_d), t)
LE2 = LE2.simplify()

In [15]:
LE1

1.0*L_1*(-L_1*m*Derivative(\theta_1(t), (t, 2)) - L_1*Derivative(\theta_1(t), (t, 2)) - L_2*m*sin(\theta_1(t) - \theta_2(t))*Derivative(\theta_2(t), t)**2 - L_2*m*cos(\theta_1(t) - \theta_2(t))*Derivative(\theta_2(t), (t, 2)) + \omega**2*m*cos(\omega*t)*cos(\theta_1(t)) + \omega**2*cos(\omega*t)*cos(\theta_1(t)) - g*m*sin(\theta_1(t)) - g*sin(\theta_1(t)))

In [16]:
LE2

1.0*L_2*m*(L_1*sin(\theta_1(t) - \theta_2(t))*Derivative(\theta_1(t), t)**2 - L_1*cos(\theta_1(t) - \theta_2(t))*Derivative(\theta_1(t), (t, 2)) - L_2*Derivative(\theta_2(t), (t, 2)) + \omega**2*cos(\omega*t)*cos(\theta_2(t)) - g*sin(\theta_2(t)))

In [17]:
sols = smp.solve([LE1, LE2], (the1_dd, the2_dd), simplify=False, rational=False)

In [18]:
sols[the1_dd]

L_1*m*sin(\theta_1(t) - \theta_2(t))*cos(\theta_1(t) - \theta_2(t))*Derivative(\theta_1(t), t)**2/(L_1*m*cos(\theta_1(t) - \theta_2(t))**2 - L_1*m - L_1) + L_2*m*sin(\theta_1(t) - \theta_2(t))*Derivative(\theta_2(t), t)**2/(L_1*m*cos(\theta_1(t) - \theta_2(t))**2 - L_1*m - L_1) + \omega**2*m*cos(\omega*t)*cos(\theta_1(t) - \theta_2(t))*cos(\theta_2(t))/(L_1*m*cos(\theta_1(t) - \theta_2(t))**2 - L_1*m - L_1) - \omega**2*m*cos(\omega*t)*cos(\theta_1(t))/(L_1*m*cos(\theta_1(t) - \theta_2(t))**2 - L_1*m - L_1) - \omega**2*cos(\omega*t)*cos(\theta_1(t))/(L_1*m*cos(\theta_1(t) - \theta_2(t))**2 - L_1*m - L_1) + g*m*sin(\theta_1(t))/(L_1*m*cos(\theta_1(t) - \theta_2(t))**2 - L_1*m - L_1) - g*m*sin(\theta_2(t))*cos(\theta_1(t) - \theta_2(t))/(L_1*m*cos(\theta_1(t) - \theta_2(t))**2 - L_1*m - L_1) + g*sin(\theta_1(t))/(L_1*m*cos(\theta_1(t) - \theta_2(t))**2 - L_1*m - L_1)

In [20]:
a = LE1.subs([(smp.sin(the1 - the2), the1 - the2),
              (smp.cos(the1 - the2), 1),
              (smp.cos(the1), 1),
              (smp.sin(the1), the1),
              (the1, C*smp.cos(w * t)),
              (the2, C*alph*smp.cos(w * t)),
              (m, 1),
              (L2, L1),
              ]).doit().series(C, 0, 2).removeO().simplify()

b = LE2.subs([(smp.sin(the1 - the2), the1 - the2),
              (smp.cos(the1 - the2), 1),
              (smp.cos(the1), 1),
              (smp.cos(the2), 1),
              (smp.sin(the1), the1),
              (smp.sin(the2), the2),
              (the1, C * smp.cos(w * t)),
              (the2, C * alph * smp.cos(w * t)),
              (m, 1),
              (L2, L1),
              ]).doit().series(C, 0, 2).removeO().simplify()

In [ ]:
yeet = smp.solve([a.args[1], b.args[2]], (w, alph))

In [ ]:
yeet[2][0]

-sqrt(-C*g*(-2.0 + 1.4142135623731*(C**2*L_1**2 + C*L_1 + 0.5)**0.5/(C*L_1) - 1/(C*L_1))/(C*L_1 + 1.0))

In [ ]:
yeet[0][0]

-sqrt(-C*g*(-2.0 - 1.4142135623731*(C**2*L_1**2 + C*L_1 + 0.5)**0.5/(C*L_1) - 1/(C*L_1))/(C*L_1 + 1.0))

In [ ]:
smp.limit(yeet[1][0].subs(C, beta/L1).simplify(), beta, smp.oo)

3*sqrt(37935706248590)*sqrt(g/L_1)/10000000

In [ ]:
# Mengubah persamaan eksak dan memasukkan ke dalam persamaan Numerik
dz1dt_f = smp.lambdify((t, m, g, w, L1, L2, the1, the2, the1_d, the2_d), sols[the1_dd])
dthe1dt_f = smp.lambdify(the1_d, the1_d)

dz2dt_f = smp.lambdify((t, m, g, w, L1, L2, the1, the2, the1_d, the2_d), sols[the2_dd])
dthe2dt_f = smp.lambdify(the2_d, the2_d)

In [ ]:
# Mendefinisikan persamaan diferensial fungsi S
def dSdt(S, t):
    the1, z1, the2, z2 = S
    return [
        dthe1dt_f(z1),
        dz1dt_f(t, m, g, w, L1, L2, the1, the2, z1, z2),
        dthe2dt_f(z2),
        dz2dt_f(t, m, g, w, L1, L2, the1, the2, z1, z2),
    ]

In [ ]:
# Menambahkan salah satu contoh fungsi numerik untuk mendapatkan nilai
t = np.linspace(0, 20, 1000)
g = 9.81
m = 1
L1 = 20
L2 = 20
w = np.sqrt(g/L1)
ans = odeint(dSdt, y0=[0, 0, 0, 0], t=t)

NameError: name 'dthe1dt_f' is not defined

In [ ]:
plt.plot(ans.T[0])

NameError: name 'plt' is not defined